Weight - zero / random initialization
Bias - Glorot

Hyperparameters:
* No of hidden layers
* no of neurons in the hidden layer
* weight initialzation
* learning rate
* No of epoch
* Batch size
* Optimisation
* Evaluation Metrics
* activation function
* loss function

Activation Function:
* sigmoid
* step function
* tanh
* relu
* linear activation

default learning rate is 0.001
no of neurons increases - wider
no of hidden layer increases - deeper

In [1]:
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, roc_curve, auc, recall_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
## For Grid search
from keras.wrappers.scikit_learn import  KerasClassifier
from sklearn.model_selection import GridSearchCV


In [3]:
tf.__version__

'2.11.0'

In [4]:
data1 = pd.read_csv("BackOrders.csv")
data1.head(3)

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1888279,117,NaN,0,0,0,0,0,0,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,1870557,7,2.0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,1475481,258,15.0,10,10,77,184,46,132,256,...,0,0.54,0.70,0,No,No,No,Yes,No,No


In [5]:
#check data type of each attribute
data1.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

In [6]:
#deleting the sku attribute
np.size(np.unique(data1.sku, return_counts = True)[0])

61589

In [7]:
data1.drop("sku", axis = 1, inplace = True)

In [8]:
data1.isnull().sum()

national_inv            0
lead_time            3403
in_transit_qty          0
forecast_3_month        0
forecast_6_month        0
forecast_9_month        0
sales_1_month           0
sales_3_month           0
sales_6_month           0
sales_9_month           0
min_bank                0
potential_issue         0
pieces_past_due         0
perf_6_month_avg        0
perf_12_month_avg       0
local_bo_qty            0
deck_risk               0
oe_constraint           0
ppap_risk               0
stop_auto_buy           0
rev_stop                0
went_on_backorder       0
dtype: int64

In [9]:
#imputing the missing values 
data1['lead_time'] = data1['lead_time'].fillna(np.mean,inplace = True)

In [10]:
data1

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,117,None,0,0,0,0,0,0,15,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,7,None,0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,258,None,10,10,77,184,46,132,256,365,...,0,0.54,0.70,0,No,No,No,Yes,No,No
3,46,None,0,0,0,0,1,2,6,9,...,0,0.75,0.90,0,Yes,No,No,Yes,No,No
4,2,None,0,4,6,10,2,2,5,6,...,0,0.97,0.92,0,No,No,No,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,6,None,0,24,24,24,0,7,9,12,...,0,0.98,0.98,0,No,No,No,Yes,No,No
61585,130,None,0,40,80,140,18,108,230,317,...,0,0.51,0.28,0,No,No,No,Yes,No,No
61586,135,None,0,0,0,0,10,40,65,89,...,0,1.00,0.99,0,No,No,Yes,Yes,No,No
61587,63,None,0,0,0,0,452,1715,3425,4864,...,0,-99.00,-99.00,1,No,No,No,No,No,Yes


In [11]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61589 entries, 0 to 61588
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   national_inv       61589 non-null  int64  
 1   lead_time          0 non-null      object 
 2   in_transit_qty     61589 non-null  int64  
 3   forecast_3_month   61589 non-null  int64  
 4   forecast_6_month   61589 non-null  int64  
 5   forecast_9_month   61589 non-null  int64  
 6   sales_1_month      61589 non-null  int64  
 7   sales_3_month      61589 non-null  int64  
 8   sales_6_month      61589 non-null  int64  
 9   sales_9_month      61589 non-null  int64  
 10  min_bank           61589 non-null  int64  
 11  potential_issue    61589 non-null  object 
 12  pieces_past_due    61589 non-null  int64  
 13  perf_6_month_avg   61589 non-null  float64
 14  perf_12_month_avg  61589 non-null  float64
 15  local_bo_qty       61589 non-null  int64  
 16  deck_risk          615

In [12]:
cat_attr = data1.columns[data1.dtypes == 'O']

In [13]:
cat_attr

Index(['lead_time', 'potential_issue', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [14]:
num_attr = data1.columns.difference(cat_attr)

In [15]:
num_attr

Index(['forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'in_transit_qty', 'local_bo_qty', 'min_bank', 'national_inv',
       'perf_12_month_avg', 'perf_6_month_avg', 'pieces_past_due',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month'],
      dtype='object')

In [16]:
cat_attr = list(cat_attr)

In [17]:
cat_attr.remove("went_on_backorder")

In [18]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
numeric_transformer = Pipeline( steps = [
    ('scale', StandardScaler())])
categorical_transformer = Pipeline(steps=[("ohe", OneHotEncoder(handle_unknown = 'ignore'))
])
preprocessor = ColumnTransformer(transformers = [('num', numeric_transformer, num_attr),
                                                 ('cat', categorical_transformer, cat_attr)])

In [19]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scale', StandardScaler())]),
                                 Index(['forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'in_transit_qty', 'local_bo_qty', 'min_bank', 'national_inv',
       'perf_12_month_avg', 'perf_6_month_avg', 'pieces_past_due',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['lead_time', 'potential_issue', 'deck_risk',
                                  'oe_constraint', 'ppap_risk', 'stop_auto_buy',
                                  'rev_stop'])])

In [20]:
#performing train test split on the data
X,y = data1.loc[:, data1.columns!='went_on_backorder'],data1.loc[:,'went_on_backorder'] 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=123)

In [21]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [22]:
X_train

array([[ 0.05504619,  0.01563573,  0.001486  , ...,  1.        ,
         1.        ,  0.        ],
       [-0.03043134, -0.0305778 , -0.03028424, ...,  1.        ,
         1.        ,  0.        ],
       [-0.03043134, -0.03011566, -0.02996654, ...,  1.        ,
         1.        ,  0.        ],
       ...,
       [-0.03043134, -0.03030051, -0.02952176, ...,  1.        ,
         1.        ,  0.        ],
       [-0.03043134, -0.03048537, -0.03009362, ...,  1.        ,
         1.        ,  0.        ],
       [-0.03043134, -0.0305778 , -0.03028424, ...,  1.        ,
         1.        ,  0.        ]])

In [23]:
num_attr = list(num_attr)

In [24]:
type(num_attr)

list